In [6]:
from sys import exit
from Crypto.Util.number import bytes_to_long, inverse, long_to_bytes
import random
import string
from pwn import *
CHARSET = string.ascii_uppercase + string.ascii_lowercase + string.digits
from Crypto.Cipher import ChaCha20, ChaCha20_Poly1305
from Crypto.Hash import Poly1305

## **Bài toán ChaCha Slide**

### 1. Thành phần của ChaCha20-Poly1305

Thuật toán này kết hợp hai thành phần chính:

#### **ChaCha20 → Mã hóa dữ liệu**
- Đây là một **stream cipher** (mã dòng) mạnh mẽ, an toàn và nhanh chóng.
- Nó nhận vào:
  - **Khóa 256-bit** (32 byte).
  - **Nonce 96-bit** (12 byte).
  - **Counter 32-bit** (4 byte).
- ChaCha20 tạo ra một **chuỗi byte ngẫu nhiên (keystream)** và thực hiện phép **XOR với plaintext** để tạo ra ciphertext.

#### **Poly1305 → Tạo và xác thực mã MAC (Message Authentication Code)**
- Poly1305 là một thuật toán tạo mã xác thực thông điệp (**MAC**) để đảm bảo rằng dữ liệu không bị thay đổi.
- Nó nhận vào **một khóa 128-bit** và tạo ra **một tag 16 byte (128-bit)**.
- **Tag này được gửi kèm với bản mã** để kiểm tra tính toàn vẹn khi giải mã.

### 2. Giải thích mã nguồn

Dưới đây là phần phân tích mã nguồn chính:

#### **1. Khởi tạo và sinh khóa**
```python
import secrets
import hashlib
from Crypto.Cipher import ChaCha20_Poly1305

flag = open("flag.txt").read().strip()

def shasum(x):
    return hashlib.sha256(x).digest()

key = shasum(shasum(secrets.token_bytes(32) + flag.encode()))
```
**Giải thích:**
- `secrets.token_bytes(32)`: Sinh một chuỗi ngẫu nhiên 32 byte.
- `flag.encode()`: Chuyển đổi nội dung của `flag.txt` thành bytes.
- `shasum()`: Hàm băm SHA-256 hai lần để tạo **khóa 256-bit** an toàn.

#### **2. Sinh Nonce ngẫu nhiên**
```python
nonce = secrets.token_bytes(12)
```
**Giải thích:**
- Sinh một **nonce 12 byte (96-bit)** ngẫu nhiên cho mỗi lần mã hóa.
- Nonce **phải duy nhất** để đảm bảo an toàn.

#### **3. Hàm mã hóa**
```python
def encrypt(message):
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    ciphertext, tag = cipher.encrypt_and_digest(message)
    return ciphertext + tag + nonce
```
**Giải thích:**
- `ChaCha20_Poly1305.new(key=key, nonce=nonce)`: Tạo đối tượng mã hóa.
- `encrypt_and_digest(message)`:
  - Mã hóa `message`.
  - Sinh **MAC tag** để đảm bảo tính toàn vẹn.
- Trả về **ciphertext + tag + nonce** để dùng khi giải mã.

#### **4. Hàm giải mã**
```python
def decrypt(message_enc):
    ciphertext = message_enc[:-28]
    tag = message_enc[-28:-12]
    nonce = message_enc[-12:]
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    return plaintext
```
**Giải thích:**
- `message_enc[:-28]`: Lấy **ciphertext** từ đầu đến -28 byte cuối.
- `message_enc[-28:-12]`: Lấy **tag (16 byte)**.
- `message_enc[-12:]`: Lấy **nonce (12 byte)**.
- `decrypt_and_verify(ciphertext, tag)`:
  - Giải mã **ciphertext**.
  - Xác thực **tag** để đảm bảo dữ liệu không bị thay đổi.

#### **5. Mã hóa & Giải mã thử nghiệm**
```python
messages = [
    "Did you know that ChaCha20-Poly1305 is an authenticated encryption algorithm?",
    "That means it protects both the confidentiality and integrity of data!"
]

goal = "But it's only secure if used correctly!"

for message in messages:
    print("Plaintext: " + repr(message))
    message = message.encode()
    print("Plaintext (hex): " + message.hex())
    ciphertext = encrypt(message)
    print("Ciphertext (hex): " + ciphertext.hex())
    print()
```
**Giải thích:**
- Mã hóa hai thông điệp thử nghiệm.
- Chuyển đổi sang **hex** để hiển thị dữ liệu mã hóa.

#### **6. Nhận đầu vào từ người dùng & Giải mã**
```python
user = bytes.fromhex(input("What is your message? "))
user_message = decrypt(user)
print("User message (decrypted): " + repr(user_message))

if goal in repr(user_message):
    print(flag)
```
**Giải thích:**
- Người dùng nhập dữ liệu đã mã hóa ở dạng **hex**.
- Giải mã và kiểm tra xem thông điệp có chứa câu `"But it's only secure if used correctly!"` hay không.
- Nếu đúng → In ra **flag**.

---
### **3. Tóm tắt**
- **ChaCha20-Poly1305** là một thuật toán mã hóa được xác thực (**AEAD**).
- **ChaCha20** để mã hóa, **Poly1305** để xác thực tính toàn vẹn.
- **Nonce 12 byte** giúp đảm bảo tính an toàn (không được trùng lặp).
- **Kết hợp mã hóa & xác thực** giúp tránh tấn công sửa đổi dữ liệu.
- **Lưu ý:** Nếu nonce bị lặp lại, dữ liệu có thể bị tấn công.

📌 **Ứng dụng thực tế:** ChaCha20-Poly1305 được dùng trong **TLS 1.3**, **WireGuard VPN**, **Google QUIC**, v.v.

---



Plaintext: 'Did you know that ChaCha20-Poly1305 is an authenticated encryption algorithm?'

Plaintext (hex): 44696420796f75206b6e6f7720746861742043686143686132302d506f6c793133303520697320616e2061757468656e7469636174656420656e6372797074696f6e20616c676f726974686d3f

Ciphertext (hex): 4409dfba5e1543bfb845547f0fd69002d943760163394820073a595db852221b9afcf218e54161fe528a5dbd820f1cb1eb35bbc77fe66cdb81107f6f4f552590a2f3faefca0d5e14b4c21a59b855fdb6013b212fa70643a3e7bd6b82c47bb6d444b0bd8969c6404331


Plaintext: 'That means it protects both the confidentiality and integrity of data!'

Plaintext (hex): 54686174206d65616e732069742070726f746563747320626f74682074686520636f6e666964656e7469616c69747920616e6420696e74656772697479206f66206461746121

Ciphertext (hex): 5408daee071753febd581b615b828811c217500a760900235a7e1c2da3563e0acaa3a95ee55624f148c35da49f1300fffe32bc8662ed7c9e830c75694f053e9fedf9bbfac74b9123572f8ccae8412719da26c5329a757bb6d444b0bd8969c6404331


What is your message? 

In [90]:
plain_lst = [
    '54686174206d65616e732069742070726f746563747320626f74682074686520636f6e666964656e7469616c69747920616e6420696e74656772697479206f66206461746121',
]
cipher_lst = [
    '0614c67712c32e2d19fd60ede85edd57d566bdd40ffcb392280be984980e3e949c9fe1d5be63ec45902ad126d32b1d0015e29ce2e6d3dbc026c6210f738ea6e4ff41e5a332bad4ae06fee783391b5b29be9bc17fc1e75d94bc5b64e87657df83df03',
]
plain_text = bytes.fromhex(plain_lst[0])
cipher_temp = bytes.fromhex(cipher_lst[0])
cipher_text = cipher_temp[:-28]
tag = cipher_temp[-28:-12]
nonce = cipher_temp[-12:]

assert(len(plain_text) == len(cipher_text))
keystream = bytes([p ^ c for p, c in zip(plain_text, cipher_text)])

plain = "But it's only secure if used correctly! ow that ChaCha20-Poly1305 isdk"
plain_bytes = plain.encode()
cipher = bytes([p ^ k for p, k in zip(plain_bytes, keystream)])

# poly1305_mac = Poly1305.new(key=keystream[:32], cipher=ChaCha20, nonce=nonce)
# poly1305_mac.update(cipher)
# computed_tag = poly1305_mac.digest()

poly1305_mac = Poly1305.new(key=keystream[:32], cipher=ChaCha20, nonce=nonce)
poly1305_mac.update(cipher_text)
print(poly1305_mac.hexdigest())
print(tag.hex())

# res = cipher + computed_tag + nonce
# # print(computed_tag)
# print(len(plain), len(cipher), len(computed_tag), len(nonce), len(keystream))
# print(res.hex())


c79702ed7f39b755428e11f8d8a70aa5
d4ae06fee783391b5b29be9bc17fc1e7


In [ ]:
# Lời giải hoàn chỉnh:
plain_lst = []
cipher_lst = []

port = 49585
r = remote('verbal-sleep.picoctf.net', port)

get_success = r.recvuntil("to guess it:  ").decode()
cipher_text = r.recvline(keepends=False).strip().decode()
# r.sendline('1337')

get_success = r.recvuntil("What would you like to do?").decode()
get_success = r.recvline(keepends=False).strip().decode()